In [1]:
filename = "../tsplib/1_test_martin_2.tsp"
distance_matrix = []

with open(filename, 'r') as f:
    lines = f.readlines()
    read_distances = False  

    for line in lines: 
        line = line.strip()
        line = line.replace(':', ' ')

        if line.startswith('EOF'):
            break
        elif line.startswith('EDGE_WEIGHT_SECTION'):
            read_distances = True
        elif read_distances:
            elements = line.split()
            distance_matrix.append([int(e) for e in elements])

distance_matrix

[[0,
  633,
  257,
  91,
  412,
  150,
  80,
  134,
  259,
  505,
  353,
  324,
  70,
  211,
  268,
  246,
  121],
 [633,
  0,
  390,
  661,
  227,
  488,
  572,
  530,
  555,
  289,
  282,
  638,
  567,
  466,
  420,
  745,
  518],
 [257,
  390,
  0,
  228,
  169,
  112,
  196,
  154,
  372,
  262,
  110,
  437,
  191,
  74,
  53,
  472,
  142],
 [91,
  661,
  228,
  0,
  383,
  120,
  77,
  105,
  175,
  476,
  324,
  240,
  27,
  182,
  239,
  237,
  84],
 [412,
  227,
  169,
  383,
  0,
  267,
  351,
  309,
  338,
  196,
  61,
  421,
  346,
  243,
  199,
  528,
  297],
 [150,
  488,
  112,
  120,
  267,
  0,
  63,
  34,
  264,
  360,
  208,
  329,
  83,
  105,
  123,
  364,
  35],
 [80, 572, 196, 77, 351, 63, 0, 29, 232, 444, 292, 297, 47, 150, 207, 332, 29],
 [134,
  530,
  154,
  105,
  309,
  34,
  29,
  0,
  249,
  402,
  250,
  314,
  68,
  108,
  165,
  349,
  36],
 [259,
  555,
  372,
  175,
  338,
  264,
  232,
  249,
  0,
  495,
  352,
  95,
  189,
  326,
  383,
  202,
  2

In [6]:
import sys

class TSPSolver:
    def __init__(self, graph):
        self.n = len(graph)
        self.graph = graph
        self.best_cost = sys.maxsize
        self.best_path = []

    def tsp(self, curr_bound, curr_cost, level, path, visited):
        if level == self.n:
            last_to_first = self.graph[path[level - 1]][path[0]]
            if last_to_first:
                final_cost = curr_cost + last_to_first
                if final_cost < self.best_cost:
                    self.best_cost = final_cost
                    self.best_path = path[:]
            return

        for i in range(self.n):
            if self.graph[path[level - 1]][i] and not visited[i]:
                temp_bound = curr_bound
                curr_cost += self.graph[path[level - 1]][i]

                if curr_cost < self.best_cost:
                    path[level] = i
                    visited[i] = True
                    self.tsp(curr_bound, curr_cost, level + 1, path, visited)

                curr_cost -= self.graph[path[level - 1]][i]
                visited[i] = False

    def solve(self):
        path = [-1] * self.n
        visited = [False] * self.n
        path[0] = 0
        visited[0] = True
        self.tsp(0, 0, 1, path, visited)
        return self.best_cost, self.best_path

# Example usage
graph = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

solver = TSPSolver(distance_matrix)
cost, path = solver.solve()
print("Minimum Cost:", cost)
print("Optimal Path:", path + [path[0]])  # Return to start

Minimum Cost: 13
Optimal Path: [0, 1, 2, 3, 4, 0]


In [19]:
import sys

class TSPSolver:
    def __init__(self, graph):
        self.n = len(graph)
        self.graph = graph  # Keep it as a regular 2D list
        self.best_cost = sys.maxsize
        self.best_path = []

    def tsp(self, curr_cost, level, path, visited):
        if level == self.n:
            last_to_first = self.graph[path[level - 1]][path[0]]
            if last_to_first > 0:  # Ensure valid edge
                final_cost = curr_cost + last_to_first
                if final_cost < self.best_cost:
                    self.best_cost = final_cost
                    self.best_path = path[:]
            return

        for i in range(self.n):
            if self.graph[path[level - 1]][i] > 0 and not visited[i]:  # Ensure valid path
                curr_cost += self.graph[path[level - 1]][i]

                if curr_cost < self.best_cost:
                    path[level] = i
                    visited[i] = True
                    self.tsp(curr_cost, level + 1, path, visited)

                curr_cost -= self.graph[path[level - 1]][i]
                visited[i] = False

    def solve_bnb(self):
        path = [-1] * self.n
        visited = [False] * self.n
        path[0] = 0
        visited[0] = True
        self.tsp(0, 1, path, visited)
        return self.best_cost, self.best_path



# Solving TSP using Branch and Bound
bnb_solver = TSPSolver(distance_matrix)
bnb_cost, bnb_path = bnb_solver.solve_bnb()
print("Branch and Bound - Minimum Cost:", bnb_cost)
print("Branch and Bound - Optimal Path:", bnb_path + [bnb_path[0]])  # Return to start

KeyboardInterrupt: 

In [18]:
class NearestNeighborSolver:
    def __init__(self, graph):
        self.graph = graph
        self.n = len(graph)

    def nearest_neighbor(self):
        visited = [False] * self.n
        path = [0]
        visited[0] = True
        total_cost = 0

        for _ in range(self.n - 1):
            last_visited = path[-1]
            nearest = None
            min_dist = sys.maxsize

            for i in range(self.n):
                if not visited[i] and 0 < self.graph[last_visited][i] < min_dist:
                    nearest = i
                    min_dist = self.graph[last_visited][i]

            path.append(nearest)
            visited[nearest] = True
            total_cost += min_dist

        total_cost += self.graph[path[-1]][path[0]]  # Return to start
        return total_cost, path
    

# Solving TSP using Nearest Neighbor
nn_solver = NearestNeighborSolver(distance_matrix)
nn_cost, nn_path = nn_solver.nearest_neighbor()
print("\nNearest Neighbor - Minimum Cost:", nn_cost)
print("Nearest Neighbor - Path:", nn_path + [nn_path[0]])  # Return to start


Nearest Neighbor - Minimum Cost: 2187
Nearest Neighbor - Path: [0, 12, 3, 6, 7, 5, 16, 13, 14, 2, 10, 4, 9, 1, 8, 11, 15, 0]


In [ ]:
import random

class NearestNeighborSolver:
    def __init__(self, graph):
        self.graph = graph
        self.n = len(graph)

    def nearest_neighbor(self, start=None):
        if start is None:
            start = random.randint(0, self.n - 1)  # Random starting city
        
        visited = [False] * self.n
        path = [start]
        visited[start] = True
        total_cost = 0

        for _ in range(self.n - 1):
            last_visited = path[-1]
            nearest = None
            min_dist = sys.maxsize

            for i in range(self.n):
                if not visited[i] and 0 < self.graph[last_visited][i] < min_dist:
                    nearest = i
                    min_dist = self.graph[last_visited][i]

            path.append(nearest)
            visited[nearest] = True
            total_cost += min_dist

        total_cost += self.graph[path[-1]][path[0]]  # Return to start
        return total_cost, path
    
# Solve using Nearest Neighbor with a random start
nn_solver = NearestNeighborSolver(distance_matrix)
nn_cost, nn_path = nn_solver.nearest_neighbor()

print("\nNearest Neighbor (Random Start) - Minimum Cost:", nn_cost)
print("Nearest Neighbor (Random Start) - Path:", nn_path + [nn_path[0]]) 


Nearest Neighbor (Random Start) - Minimum Cost: 2187
Nearest Neighbor (Random Start) - Path: [0, 12, 3, 6, 7, 5, 16, 13, 14, 2, 10, 4, 9, 1, 8, 11, 15, 0]


In [16]:
import sys

class TwoOptSolver:
    def __init__(self, graph):
        self.graph = graph
        self.n = len(graph)

    def calculate_cost(self, path):
        cost = sum(self.graph[path[i]][path[i+1]] for i in range(self.n - 1))
        cost += self.graph[path[-1]][path[0]]  # Return to start
        return cost

    def two_opt(self, path):
        best_path = path[:]
        best_cost = self.calculate_cost(best_path)
        improved = True

        while improved:
            improved = False
            for i in range(1, self.n - 1):
                for j in range(i + 1, self.n):
                    new_path = best_path[:i] + best_path[i:j+1][::-1] + best_path[j+1:]
                    new_cost = self.calculate_cost(new_path)

                    if new_cost < best_cost:
                        best_path, best_cost = new_path, new_cost
                        improved = True
        
        return best_cost, best_path

# Solving TSP using 2-Opt (starting from Nearest Neighbor solution)
two_opt_solver = TwoOptSolver(distance_matrix)
opt_cost, opt_path = two_opt_solver.two_opt(nn_path)
print("\n2-Opt - Optimized Cost:", opt_cost)
print("2-Opt - Optimized Path:", opt_path + [opt_path[0]])




2-Opt - Optimized Cost: 2085
2-Opt - Optimized Path: [0, 3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 9, 1, 4, 8, 11, 15, 0]


In [10]:
import networkx as nx

class ChristofidesSolver:
    def __init__(self, graph):
        self.graph = graph
        self.n = len(graph)
    
    def christofides(self):
        G = nx.Graph()
        for i in range(self.n):
            for j in range(i + 1, self.n):
                G.add_edge(i, j, weight=self.graph[i][j])

        # Step 1: Minimum Spanning Tree (MST)
        MST = nx.minimum_spanning_tree(G)

        # Step 2: Find odd-degree vertices
        odd_vertices = [v for v in MST.nodes if MST.degree[v] % 2 == 1]

        # Step 3: Minimum Weight Perfect Matching
        odd_graph = nx.Graph()
        for i in range(len(odd_vertices)):
            for j in range(i + 1, len(odd_vertices)):
                u, v = odd_vertices[i], odd_vertices[j]
                odd_graph.add_edge(u, v, weight=self.graph[u][v])

        matching = nx.min_weight_matching(odd_graph, maxcardinality=True)

        # Step 4: Combine MST and Matching
        multigraph = nx.MultiGraph(MST)
        for u, v in matching:
            multigraph.add_edge(u, v, weight=self.graph[u][v])

        # Step 5: Eulerian Circuit → Hamiltonian Cycle
        eulerian_circuit = list(nx.eulerian_circuit(multigraph))
        path = []
        visited = set()

        for u, v in eulerian_circuit:
            if u not in visited:
                path.append(u)
                visited.add(u)

        path.append(path[0])  # Return to start
        total_cost = sum(self.graph[path[i]][path[i+1]] for i in range(self.n))
        return total_cost, path

# Solving TSP using Christofides Algorithm
christofides_solver = ChristofidesSolver(distance_matrix)
chris_cost, chris_path = christofides_solver.christofides()
print("\nChristofides - Approximate Cost:", chris_cost)
print("Christofides - Approximate Path:", chris_path)


Christofides - Approximate Cost: 2197
Christofides - Approximate Path: [0, 15, 11, 8, 3, 12, 6, 16, 13, 14, 2, 10, 9, 1, 4, 5, 7, 0]
